In [1]:
# !pip uninstall tensorflow-gpu -y --user
!pip install tensorflow==1.13.1

# Data upload

In [2]:
!mkdir .kaggle

mkdir: cannot create directory ‘.kaggle’: File exists


In [6]:
import json
token = {"username": "trokhymovych", "key": "071d0c0ef7f58991684890d02542875e"}
with open('.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [9]:
!cp .kaggle/kaggle.json ~/.kaggle/kaggle.json

In [12]:
!kaggle config set -n path -v{/content}

- path is now set to: {/content}


In [13]:
!chmod 600 ~/.kaggle/kaggle.json

In [14]:
!kaggle competitions download -c evohackaton

100%|█████████████████████████████████████▉| 2.67G/2.67G [02:07<00:00, 26.6MB/s]
100%|██████████████████████████████████████| 2.67G/2.67G [02:07<00:00, 22.5MB/s]


In [ ]:
!unzip {/content}/competitions/evohackaton/evohackaton.zip

# Data processing

In [2]:
import os
import keras
import glob
import keras
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
from sklearn.model_selection import train_test_split
from keras.models import load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Model 
from tensorflow.keras.models import Sequential 
from keras import layers
from keras.layers import Conv2D,MaxPooling2D, BatchNormalization, Dropout, GlobalAveragePooling2D, Dense, Flatten
from keras import optimizers
from keras.applications.inception_v3 import InceptionV3
from keras.regularizers import l2

Using TensorFlow backend.


In [3]:
# Set general direction
dir = ''

In [4]:
# Create a dictionary from the  data file: {img.jpg : lable}

# Go to label directiory and tranfer labels to pd
labels = pd.read_csv(dir + 'train.csv').set_index('name')

# Convert pd to dic -> {Image.jpg : label}
labels = labels.to_dict()
labels = labels[list(labels.keys())[0]]

print(f'lenght of dictionary: {len(labels)}')

lenght of dictionary: 16857


In [5]:
# Create a PD dataframe with columns Images and Labels
# This DF is needed for the flow_from_dataframe method of the ImageDataGenerator class
df = pd.DataFrame(list(labels.items()))
df.columns = ['images', 'labels']
df = df.astype({'labels': str})

# Display the occurance of each class
df.groupby('labels').count().head()

,images
labels,
0,251
1,155
10,89
11,187
12,131


In [6]:
train_df, validation_df = train_test_split(df, test_size=0.1)

print(f'train_df: {train_df.shape}')
print(f'validation_df: {validation_df.shape}')

train_df: (15171, 2)
validation_df: (1686, 2)


In [7]:
train_df.head()

,images,labels
14999,14999.jpg,39
11636,11636.jpg,42
10728,10728.jpg,0
7829,7829.jpg,57
179,179.jpg,14


In [8]:
# Preprocessing settings for training data
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    rotation_range=15,
    height_shift_range=0.1,
    width_shift_range=0.1,
    shear_range=0.01,
    zoom_range=[0.9, 1.25],
    brightness_range=[0.5, 1.5],
    horizontal_flip=True,
    vertical_flip=False,
    fill_mode='reflect')

# Preprocessing settings for test data
validation_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255)

In [9]:
# Apply data transfers and create generators for training and test data
train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory='train/train',
        x_col="images",
        y_col="labels",
        target_size=(256, 256),
        batch_size=32,
        class_mode='categorical')

validation_generator = validation_datagen.flow_from_dataframe(
        dataframe=validation_df,
        directory='train/train',
        x_col="images",
        y_col="labels",
        target_size=(256, 256),
        batch_size=32,
        class_mode='categorical')

Found 15171 validated image filenames belonging to 100 classes.
Found 1686 validated image filenames belonging to 100 classes.


In [ ]:
# Cell below is used to initiate a cnn network
# Specify the shape of inputs
input_shape = (256,256,3)
batch_size = 32
n_classes = 100

# Set up model
base_model = InceptionV3(weights='imagenet', include_top=False, input_tensor=keras.layers.Input(shape=input_shape))

for layer in base_model.layers:
      layer.trainable = False
        
x = base_model.output
x = keras.layers.AveragePooling2D(pool_size=(6, 6))(x)
x = keras.layers.Dropout(.4)(x)
x = keras.layers.Flatten()(x)

predictions = keras.layers.Dense(n_classes, init='glorot_uniform', W_regularizer=l2(.0005), activation='softmax')(x)
model = Model(input=base_model.input, output=predictions)
# Display model
model.summary()

# Show if GPU is avialable
# print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Instructions for updating:
Colocations handled automatically by placer.
38092800/87910968 [===========>..................] - ETA: 48s

In [ ]:
# Initaite mode, specify the optimizer, lossfunction and metrics
opt = keras.optimizers.SGD(lr=.01, momentum=.9)

model.compile(optimizer = opt,
              loss ='categorical_crossentropy',
              metrics = ['accuracy'])

# Save weights if model improved
filepath = "weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

# Early stopping in case val_loss < min_delta for a specific number of runs 
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.03, patience=5, verbose=1, mode='auto', baseline=None, restore_best_weights=False)

# Set callbacks 
callbacks_list = [checkpoint, early_stopping]

In [ ]:
# same model as whole
model.save("Inception_transfer_10epocs.h5")

# Fit data & train model
model.fit_generator(
        train_generator,
        steps_per_epoch = train_generator.samples // batch_size,
        callbacks = callbacks_list,
        epochs = 1,
        validation_data = validation_generator,
        validation_steps = validation_generator.samples // batch_size)

In [ ]:
# same model as whole
model.save("Inception_transfer_10_epocs.h5")